# ejecutar solo cuando todos los archivo de una fecha esten en la nube

In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import storage as gcs

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "MY_CREDENTIALS"
client_gcs = gcs.Client()

bucket = client_gcs.bucket('MY_BUCKET')

In [3]:
path = "../ORIGEN/"
path_original =  path+'originales/'
path_procesados =  path+'procesados/'
path

'/home/datos_covid/'

### DESCARGANDO ARCHIVOS PENDIENTES

In [ ]:
# DESCARGAR INFECTADOS
count = 0
for elem in descargar_infectado:
    blob = bucket.blob(elem[0])
    blob.download_to_filename(path_original+elem[1])
    count += 1
    print("Elemento",count,":",path_original+elem[1])

In [5]:
descargar_infectado = []
descargar_records = []
for b in bucket.list_blobs():
    nombre = b.name
    try:
        data = nombre.split('/FILES/')
        
        if len(data) == 2 and data[1][-4:] == '.csv':
            if data[0] == 'infected':
                nombre_file = +data[1]
                if not nombre_file in descargados:
                    descargar_infectado.append([nombre,nombre_file])
            
            elif data[0] == 'records':
                nombre_file = data[1]
                if not nombre_file in descargados:
                    descargar_records.append([nombre,nombre_file])
    except:
        pass
    
print(len(descargar_infectado),len(descargar_records))

0 4


In [4]:
descargados = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path_original):
    for file in f:
        if '.csv' in file:
            descargados.append(file)

In [ ]:
# DESCARGAR INFECTADOS
count = 0
for elem in descargar_infectado:
    blob = bucket.blob(elem[0])
    blob.download_to_filename(path_original+elem[1])

### ORDENANDO ARCHIVOS PENDIENTES - LOCALIZACION

In [8]:
temp = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path_original):
    for file in f:
        if '.csv' in file:
            temp.append(file)
            
descargados = []
for elem in temp:
    if elem[0:4]=='gps_':
        descargados.append(elem)
        
descargados_fechas = []
for elem in descargados:
    descargados_fechas.append(elem[0:12])
    
descargados_fechas = np.unique(descargados_fechas)

In [ ]:
temp = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path_procesados):
    for file in f:
        if '.csv' in file:
            temp.append(file)
            
procesados = []
for elem in temp:
    if elem[0:4]=='gps_':
        procesados.append(elem)
        
procesados_fechas = []
for elem in procesados:
    procesados_fechas.append(elem[0:12])
    
procesados_fechas = np.unique(procesados_fechas)


In [ ]:
pendientes_fecha = []
for elem in descargados_fechas:
    if not elem in procesados_fechas:
        pendientes_fecha.append(elem)
        
pendientes_fecha

In [ ]:
for fecha in pendientes_fecha:
    temporales_fecha = []
    for elem in descargados:
        if fecha == elem[0:12]:
            temporales_fecha.append(elem)
    #print(temporales_fecha)
    
    numero_lotes = len(temporales_fecha)
    
    print("### ",fecha," ###")
    
    print("Leyendo...")
    datos = pd.read_csv(path_original+temporales_fecha[0])
    print("   ",temporales_fecha[0])
    for file in temporales_fecha[1:]:
        data_temp = pd.read_csv(path_original+file)
        datos = pd.concat([datos,data_temp])
        print("   ",file)
    
    print("Procesando...")
    print("   ","archivos:",numero_lotes)
    datos = datos.sort_values(by=['datetime']).reset_index(drop=True)   
    rangos = []
    count = 0
    limite = int(len(datos)/numero_lotes)+1
    for i in range(numero_lotes):
        rangos.append([i,count,count+limite])
        count = count+limite
    
    
    print("Guardando...")
    for elem in rangos:
        datos_temp = datos[(datos.index >= elem[1]) & (datos.index < elem[2])]

        numero_file = 0
        if elem[0]<10: numero_file = '0'+str(elem[0])
        else: numero_file = str(elem[0])

        path_temp = path_procesados+fecha+'_lote_'+numero_file+'.csv'
        print("   ",path_temp)
        datos_temp.to_csv(path_temp, index=False)
        
print("Terminado.../")